In [1]:
import tensorflow as tf
import os
import random
import shutil

tf.random.set_seed(1)
random.seed(1)

In [6]:
PATH ='./data/dogs-vs-cats/'

train_dir = os.path.join(PATH, 'train')
valid_dir = os.path.join(PATH, 'validation')
test_dir = os.path.join(PATH, 'test')

In [7]:
train_len = len(os.listdir(train_dir))
valid_len = len(os.listdir(valid_dir))
test_len = len(os.listdir(test_dir))

In [8]:
print('total training images:', train_len)
print('total validation images:', valid_len)
print('total testing images:', test_len)

total training images: 20000
total validation images: 5000
total testing images: 12500


In [9]:
# def create_validation_sets():
#     shutil.rmtree(valid_dir)
#     os.mkdir(valid_dir)
#     shutil.rmtree(train_dir)
#     os.mkdir(train_dir)
    
#     valid_images = random.sample(os.listdir(original_dir), int(train_len * 0.20))
    
#     for image in valid_images:
#         shutil.move(os.path.join(original_dir, image), valid_dir)

# create_validation_sets()

In [52]:
CLASSNAMES = ["cat", "dog"]

In [63]:
def process_data(path):
    label = tf.strings.split(path, os.path.sep)[-1]
    label = tf.strings.split(label, ".")[0]
    return tf.cast(label == CLASSNAMES, tf.float32)

def input_fn(path):
    list_ds = tf.data.Dataset.list_files(str(path))
    return list_ds.map(process_data)

In [64]:
train = input_fn(os.path.join(train_dir, "*"))

In [65]:
for i in train.take(5):
    print(i)

tf.Tensor([0. 1.], shape=(2,), dtype=float32)
tf.Tensor([1. 0.], shape=(2,), dtype=float32)
tf.Tensor([0. 1.], shape=(2,), dtype=float32)
tf.Tensor([0. 1.], shape=(2,), dtype=float32)
tf.Tensor([1. 0.], shape=(2,), dtype=float32)
